# Baseline Experiment - Santa 2025 Christmas Tree Packing

Establish baseline by evaluating available pre-optimized submissions and creating an ensemble.

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import os

getcontext().prec = 25
scale_factor = Decimal('1e15')

In [ ]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

In [ ]:
def has_overlap(trees):
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False
    
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False


def get_side_length(trees):
    """Get the side length of the bounding box for a list of trees."""
    if not trees:
        return Decimal('0')
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    width = maxx - minx
    height = maxy - miny
    return max(width, height)


def calculate_score(side_lengths):
    """Calculate the total score from side lengths dict."""
    score = Decimal('0')
    for n, side in side_lengths.items():
        score += side ** 2 / Decimal(str(n))
    return float(score)

In [ ]:
def load_submission(csv_path):
    """Load a submission CSV and return dict of tree lists and side lengths."""
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.lstrip('s')
    df['y'] = df['y'].astype(str).str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.lstrip('s')
    
    tree_lists = {}
    side_lengths = {}
    
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        group = df[df['id'].str.startswith(prefix)]
        trees = [ChristmasTree(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
        tree_lists[n] = trees
        side_lengths[n] = get_side_length(trees)
    
    return tree_lists, side_lengths

In [ ]:
# Evaluate sample submission
print("Evaluating sample submission...")
sample_trees, sample_sides = load_submission('/home/code/preoptimized/sample_submission.csv')
sample_score = calculate_score(sample_sides)
print(f"Sample submission score: {sample_score:.6f}")

In [ ]:
# Evaluate saspav_best submission
print("\nEvaluating saspav_best submission...")
saspav_trees, saspav_sides = load_submission('/home/code/preoptimized/saspav_best.csv')
saspav_score = calculate_score(saspav_sides)
print(f"saspav_best score: {saspav_score:.6f}")

# Check for overlaps
overlap_count = 0
for n in range(1, 201):
    if has_overlap(saspav_trees[n]):
        overlap_count += 1
print(f"Overlapping N values: {overlap_count}")

In [ ]:
# Evaluate best_snapshot submission
print("\nEvaluating best_snapshot submission...")
snapshot_trees, snapshot_sides = load_submission('/home/code/preoptimized/best_snapshot.csv')
snapshot_score = calculate_score(snapshot_sides)
print(f"best_snapshot score: {snapshot_score:.6f}")

# Check for overlaps
overlap_count = 0
for n in range(1, 201):
    if has_overlap(snapshot_trees[n]):
        overlap_count += 1
print(f"Overlapping N values: {overlap_count}")

In [ ]:
# Create ensemble - pick best valid solution for each N
print("\nCreating ensemble...")

ensemble_trees = {}
ensemble_sides = {}
source_counts = {'sample': 0, 'saspav': 0, 'snapshot': 0}

for n in range(1, 201):
    candidates = []
    
    # Sample submission
    if not has_overlap(sample_trees[n]):
        candidates.append(('sample', sample_trees[n], sample_sides[n]))
    
    # Saspav submission
    if not has_overlap(saspav_trees[n]):
        candidates.append(('saspav', saspav_trees[n], saspav_sides[n]))
    
    # Snapshot submission
    if not has_overlap(snapshot_trees[n]):
        candidates.append(('snapshot', snapshot_trees[n], snapshot_sides[n]))
    
    if candidates:
        # Pick the one with smallest side length
        best = min(candidates, key=lambda x: x[2])
        ensemble_trees[n] = best[1]
        ensemble_sides[n] = best[2]
        source_counts[best[0]] += 1
    else:
        # Fallback to sample (should always be valid)
        ensemble_trees[n] = sample_trees[n]
        ensemble_sides[n] = sample_sides[n]
        source_counts['sample'] += 1

ensemble_score = calculate_score(ensemble_sides)
print(f"Ensemble score: {ensemble_score:.6f}")
print(f"Source distribution: {source_counts}")

In [ ]:
# Save ensemble submission
print("\nSaving ensemble submission...")

rows = []
for n in range(1, 201):
    trees = ensemble_trees[n]
    for t_idx, tree in enumerate(trees):
        rows.append({
            'id': f'{n:03d}_{t_idx}',
            'x': f's{float(tree.center_x):.12f}',
            'y': f's{float(tree.center_y):.12f}',
            'deg': f's{float(tree.angle):.12f}'
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/code/experiments/001_baseline/submission.csv', index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved submission with {len(submission_df)} rows")
print(f"\nFinal ensemble score: {ensemble_score:.6f}")
print(f"Target score: 68.901319")
print(f"Gap to target: {ensemble_score - 68.901319:.6f}")

In [ ]:
# Analyze score breakdown by N ranges
print("\nScore breakdown by N ranges:")
ranges = [(1, 10), (11, 20), (21, 50), (51, 100), (101, 150), (151, 200)]

for start, end in ranges:
    range_score = sum(float(ensemble_sides[n]**2 / Decimal(str(n))) for n in range(start, end+1))
    print(f"N={start:3d}-{end:3d}: {range_score:.6f}")

In [ ]:
# Show top 10 worst N values (highest contribution to score)
print("\nTop 10 worst N values (highest score contribution):")
contributions = [(n, float(ensemble_sides[n]**2 / Decimal(str(n)))) for n in range(1, 201)]
contributions.sort(key=lambda x: x[1], reverse=True)

for n, contrib in contributions[:10]:
    print(f"N={n:3d}: contribution={contrib:.6f}, side_length={float(ensemble_sides[n]):.6f}")